In [1]:
# 1 Importing Dependencies
 
import pandas as pd
import requests
import sys
from pymongo import MongoClient
from collections import defaultdict
sys.path.append('/Users/tokar/UNC_DA/MY_WORK/Personal_Code')
from my_api_config import odds_api_key2
from datetime import datetime


In [2]:
# 2 Code to get list of game dates from all games

# Establishing MongoDB Connection, to 'nba_data' database and 'games' collection
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]
collection = db["games_cleaned"]

# Query to fetch all documents
query = {}

# Projecttion to include on the game ids and not the "_id" from the documents
projection = {'date_start':1, '_id': 0}

# Execute the query and get a cursor for the results
cursor = collection.find(query, projection)

# Extract the 'date_start' values from each document in the cursor
game_dates = [doc['date_start'] for doc in cursor]

# Remove duplicates by converting the list to a set and then back to a list
game_dates = list(set(game_dates))

# Convert the list of game IDs to a DataFrame
model_df = pd.DataFrame(game_dates, columns=['game_dates'])

# Convert the DataFrame column to a list
game_dates = model_df['game_dates'].tolist()

# Close the MongoDB connection
mongo.close()

# Printing list and count of games in list
print(f"{len(game_dates)} games in game_dates")
print(game_dates)

2888 games in game_dates
['2023-03-29T23:00:00.000Z', '2021-12-31T00:00:00.000Z', '2023-11-18T00:30:00.000Z', '2024-01-14T01:00:00.000Z', '2023-02-17T01:00:00.000Z', '2021-08-09T19:00:00.000Z', '2022-07-13T20:00:00.000Z', '2021-12-09T03:00:00.000Z', '2023-03-09T00:30:00.000Z', '2023-10-15T22:00:00.000Z', '2022-01-18T00:30:00.000Z', '2022-04-14T01:30:00.000Z', '2023-04-08T02:00:00.000Z', '2023-11-07T00:00:00.000Z', '2023-11-19T23:00:00.000Z', '2022-07-15T21:00:00.000Z', '2021-08-16T02:00:00.000Z', '2024-04-12T23:30:00.000Z', '2022-02-28T00:30:00.000Z', '2023-12-24T00:00:00.000Z', '2022-05-01T00:30:00.000Z', '2022-07-14T00:00:00.000Z', '2024-02-25T23:00:00.000Z', '2022-03-27T00:00:00.000Z', '2022-02-11T00:00:00.000Z', '2023-01-10T00:30:00.000Z', '2022-07-12T02:00:00.000Z', '2021-11-27T03:00:00.000Z', '2023-12-17T00:00:00.000Z', '2023-03-27T23:00:00.000Z', '2024-01-24T03:00:00.000Z', '2022-04-19T02:00:00.000Z', '2022-11-23T01:00:00.000Z', '2022-04-22T01:00:00.000Z', '2022-12-06T00:30:00.0

In [3]:
# 3 Need to truncate time to 00:00:00, remove the duplicates and sort

# Function to truncate time
def truncate_time(dates):
    truncated_dates = []
    for date_str in dates:
        date_obj = datetime.fromisoformat(date_str.replace('Z', '+00:00'))
        truncated_date_str = date_obj.strftime('%Y-%m-%dT00:00:00Z')
        truncated_dates.append(truncated_date_str)
    return truncated_dates

# Truncate times
truncated_game_dates = truncate_time(game_dates)

# Removing dupicates
truncated_game_dates = list(set(truncated_game_dates))

# Sorting dates
truncated_game_dates = sorted(truncated_game_dates)

print(f"{len(truncated_game_dates)} games in truncated_game_dates")
print(truncated_game_dates)

735 games in truncated_game_dates
['2021-08-03T00:00:00Z', '2021-08-04T00:00:00Z', '2021-08-05T00:00:00Z', '2021-08-06T00:00:00Z', '2021-08-07T00:00:00Z', '2021-08-08T00:00:00Z', '2021-08-09T00:00:00Z', '2021-08-10T00:00:00Z', '2021-08-11T00:00:00Z', '2021-08-12T00:00:00Z', '2021-08-13T00:00:00Z', '2021-08-14T00:00:00Z', '2021-08-15T00:00:00Z', '2021-08-16T00:00:00Z', '2021-08-17T00:00:00Z', '2021-08-18T00:00:00Z', '2021-10-03T00:00:00Z', '2021-10-04T00:00:00Z', '2021-10-05T00:00:00Z', '2021-10-06T00:00:00Z', '2021-10-07T00:00:00Z', '2021-10-08T00:00:00Z', '2021-10-09T00:00:00Z', '2021-10-10T00:00:00Z', '2021-10-11T00:00:00Z', '2021-10-12T00:00:00Z', '2021-10-13T00:00:00Z', '2021-10-14T00:00:00Z', '2021-10-15T00:00:00Z', '2021-10-16T00:00:00Z', '2021-10-19T00:00:00Z', '2021-10-20T00:00:00Z', '2021-10-21T00:00:00Z', '2021-10-22T00:00:00Z', '2021-10-23T00:00:00Z', '2021-10-24T00:00:00Z', '2021-10-25T00:00:00Z', '2021-10-26T00:00:00Z', '2021-10-27T00:00:00Z', '2021-10-28T00:00:00Z', '2021

In [4]:
# 4 API Calling in Historical Events information and pushing into MongoDB collection called 'odds_events'

# Establishing MongoDB Connection
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]
collection = db["odds_events"]

# Setting up API call structure for the-odds-api.com
url_base = "https://api.the-odds-api.com/v4/historical/sports/basketball_nba/events?apiKey="
commenceTimeStart = "2021-10-01T12:00:00Z"
commenceTimeEnd = "2024-06-30T12:00:00Z"

# Looping through list of dates to call API
for date in truncated_game_dates:
    end_point_base = f"{odds_api_key2}&date={date}&commenceTimeFrom={commenceTimeStart}&commenceTimeTo={commenceTimeEnd}"
    query_url = url_base + end_point_base

    response = requests.get(query_url)

    if response.status_code == 200:
        response_data = response.json()
        if 'data' in response_data:
            for event in response_data['data']:
                collection.insert_one(event)
        else:
            print(f"No data found for date: {date}")
            
    else:
        print(f"Failed to fetch data for date: {date}. Staus code: {response.status_code}")

print("Finished fetching and inserting data.")

Finished fetching and inserting data.


In [16]:
# 5 API Calling in Historical Odds Endpoint information and pushing into MongoDB collection called 'odds_odds'

# Establishing MongoDB Connection
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]
collection = db["odds_odds"]

# Setting up API call structure for the-odds-api.com
url_base = "https://api.the-odds-api.com/v4/historical/sports/basketball_nba/odds/?apiKey="

# Looping through list of dates to call API
for date in truncated_game_dates:
    end_point_base = f"{odds_api_key2}&regions=us&markets=totals&oddsFormat=american&date={date}"
    query_url = url_base + end_point_base
        
    response = requests.get(query_url)

    if response.status_code == 200:
        response_data = response.json()
        if 'data' in response_data:
            for event in response_data['data']:
                collection.insert_one(event)
        else:
            print(f"No data found for date: {date}")
            
    else:
        print(f"Failed to fetch data for date: {date}. Staus code: {response.status_code}")

print("Finished fetching and inserting data.")

Finished fetching and inserting data.


In [33]:
# 6 Creating copy of odds_odds collection to work out of

# Establishing MongoDB Connection
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]

source_collection_name = "odds_odds"
target_collection_name = "odds_odds_copy"

source_collection = db[source_collection_name]
target_collection = db[target_collection_name]

# Ensure the target collection is empty
target_collection.delete_many({})

# Copy documents from source to target
documents = list(source_collection.find())
if documents:
    target_collection.insert_many(documents)

print(f"Copied {len(documents)} documents from '{source_collection_name}' to '{target_collection_name}'.")


Copied 8959 documents from 'odds_odds' to 'odds_odds_copy'.


In [34]:
# 7 Getting list of unique bookmakers

# Establishing MongoDB Connection
client = MongoClient('mongodb://localhost:27017/')
db = client['nba_data2']
collection = db['odds_odds_copy']

# Step 1: Use aggregation to find unique bookmakers
pipeline = [
    {"$unwind": "$bookmakers"},  # Unwind the bookmakers array
    {"$group": {"_id": "$bookmakers.key"}},  # Group by the bookmaker key
    {"$project": {"_id": 0, "bookmaker": "$_id"}}  # Project the unique bookmaker keys
]

unique_bookmakers = list(collection.aggregate(pipeline))

# Step 2: Extract the unique bookmaker keys into a list
unique_bookmaker_list = [bm['bookmaker'] for bm in unique_bookmakers]

print("Unique bookmakers:", unique_bookmaker_list)

Unique bookmakers: ['draftkings', 'mybookieag', 'bovada', 'betrivers', 'lowvig', 'caesars', 'barstool', 'betus', 'intertops', 'superbook', 'circasports', 'unibet', 'pointsbetus', 'twinspires', 'betonlineag', 'unibet_us', 'wynnbet', 'gtbets', 'sugarhouse', 'foxbet', 'fanduel', 'betmgm', 'williamhill_us']


In [35]:
# 8 Deleting any documents where the bookmakers array is empty and then sorting bookmakers array

# Establishing MongoDB Connection
client = MongoClient('mongodb://localhost:27017/')
db = client['nba_data2']
collection = db['odds_odds_copy']

# Delete documents where the bookmakers array is empty
result = collection.delete_many({"bookmakers": {"$size": 0}})

print(f"Deleted {result.deleted_count} documents where 'bookmakers' array is empty.")

# Sorting bookmakers in order to prepare to 

# Order of bookmakers
bookmaker_order = [
    "draftkings", "williamhill_us", "bovada", "betonlineag", "fanduel",
    "betmgm", "pointsbetus", "lowvig", "wynnbet", "mybookieag", "betrivers",
    "twinspires", "betus", "barstool", "sugarhouse", "foxbet", "superbook",
    "unibet_us", "unibet", "gtbets", "circasports", "intertops", "caesars"
]

# Create a dictionary to map each bookmaker to its position in the order
bookmaker_rank = {bookmaker: rank for rank, bookmaker in enumerate(bookmaker_order)}

# Retrieve all documents
documents = collection.find()

# Sort the bookmakers array in each document
for doc in documents:
    if 'bookmakers' in doc:
        sorted_bookmakers = sorted(doc['bookmakers'], key=lambda x: bookmaker_rank.get(x['key'], float('inf')))
        collection.update_one(
            {'_id': doc['_id']},
            {'$set': {'bookmakers': sorted_bookmakers}}
        )

print("Bookmakers in all documents have been sorted.")

Deleted 1699 documents where 'bookmakers' array is empty.
Bookmakers in all documents have been sorted.


In [15]:
# 9 This code removes any duplicate games based on id and checks that draftkings is the first bookmaker in the array, draftkings are the preferred bookmaker

# Establishing MongoDB Connection
client = MongoClient('mongodb://localhost:27017/')
db = client['nba_data2']
collection = db['odds_odds_copy']

# Identify duplicates based on the `id` field
pipeline = [
    {
        "$group": {
            "_id": "$id",  # Group by the `id` field
            "uniqueIds": {"$addToSet": "$_id"},
            "docs": {"$push": "$$ROOT"},
            "count": {"$sum": 1}
        }
    },
    {
        "$match": {
            "count": {"$gt": 1}  # Filter groups that have more than one document
        }
    }
]

duplicates = list(collection.aggregate(pipeline))

# Remove duplicates, keeping the one where the first bookmaker is 'draftkings' if present
for group in duplicates:
    docs = group['docs']
    ids_to_keep = []
    ids_to_delete = []

    # Check if any document has 'draftkings' as the first bookmaker
    draftkings_doc = None
    for doc in docs:
        if doc['bookmakers'] and doc['bookmakers'][0]['key'] == 'draftkings':
            draftkings_doc = doc['_id']
            break

    # If a draftkings doc is found, keep it, otherwise keep the first document
    if draftkings_doc:
        ids_to_keep.append(draftkings_doc)
    else:
        ids_to_keep.append(docs[0]['_id'])

    # Collect all other ids to delete
    ids_to_delete = [doc['_id'] for doc in docs if doc['_id'] not in ids_to_keep]

    if ids_to_delete:
        collection.delete_many({"_id": {"$in": ids_to_delete}})

print("Duplicate documents removed, keeping those with 'draftkings' as the first bookmaker where applicable.")


Duplicate documents removed, keeping those with 'draftkings' as the first bookmaker where applicable.


In [16]:
# 10 Keeping only the first bookmaker in the array

# Establishing MongoDB Connection
client = MongoClient('mongodb://localhost:27017/')
db = client['nba_data2']
collection = db['odds_odds_copy']

# Retrieve all documents
documents = collection.find()

# Update documents to keep only the first bookmaker in the array
for doc in documents:
    if 'bookmakers' in doc and len(doc['bookmakers']) > 1:
        first_bookmaker = [doc['bookmakers'][0]]  # Keep only the first bookmaker
        collection.update_one(
            {'_id': doc['_id']},
            {'$set': {'bookmakers': first_bookmaker}}
        )

print("Updated documents to keep only the first bookmaker in the array.")


Updated documents to keep only the first bookmaker in the array.


In [21]:
# 11 Restructing documents in odd_odds_copy collection to remove any unnecessary fields and flatten the documents, also creating Unique_key and changing Clippers name to match the 'joined_games' collection

# Establishing MongoDB Connection, to 'nba_data' database
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]

# Drop the existing 'games_cleaned' collection if it exists
if "odds_odds_cleaned" in db.list_collection_names():
    db.drop_collection("odds_odds_cleaned")
    print("Existing 'odds_odds_cleaned' collection dropped.")

# Existing collection name
existing_collection_name = "odds_odds_copy"

# New collection name
new_collection_name = "odds_odds_cleaned"

# Get the existing collection
existing_collection = db[existing_collection_name]

# New collection for documents with renamed fields
new_collection = db[new_collection_name]

# Iterate over documents in the existing collection
for document in existing_collection.find({}):
    # Extract necessary fields from the existing document
    game_id = document["id"]
    date_start = document["commence_time"]
    visitor = document["away_team"]
    home = document["home_team"]
    points = document["bookmakers"][0]["markets"][0]["outcomes"][0]["point"]
    
    # Replace "Los Angeles Clippers" with "LA Clippers" if necessary
    if home == "Los Angeles Clippers":
        home = "LA Clippers"
    if visitor == "Los Angeles Clippers":
        visitor = "LA Clippers"
    
    unique_key = date_start[:10] + home + visitor
    
    # Create a new document with the desired structure
    new_document = {
        "unique_key": unique_key,
        "game_id": game_id,
        "commence_time": date_start,
        "home_team": home,
        "visitor_team": visitor,
        "Over/Under": points,
    }

    
    # Insert the new document into the new collection
    new_collection.insert_one(new_document)

# Close MongoDB connection
mongo.close()

Existing 'odds_odds_cleaned' collection dropped.


In [25]:
# 12 Creating new collection 'Model_Ready" that merges 'joined_games' and 'odds_odds_copy' collections

# Connect to MongoDB
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]

if "Model_Ready" in db.list_collection_names():
    db.drop_collection("Model_Ready")
    print("Existing 'Model_Ready' collection dropped.")

# Define the aggregation pipeline
pipeline = [
    {
        "$lookup": {
            "from": "odds_odds_cleaned",
            "localField": "unique_key",
            "foreignField": "unique_key",
            "as": "odds_data"
        }
    },
    {
        "$unwind": {
            "path": "$odds_data",
            "preserveNullAndEmptyArrays": True
        }
    },
    {
        "$project": {
            "_id": 0,
            "_id_game": 1,
            "game_id": 1,
            "date_start": 1,
            "visitor_name": 1,
            "visitor_nickname": 1,
            "visitor_code": 1,
            "home_id": 1,
            "home_name": 1,
            "home_nickname": 1,
            "home_code": 1,
            "visitor_points_game": 1,
            "home_points_game": 1,
            "_id_stats": 1,
            "visitor_points_stats": 1,
            "visitor_fgm": 1,
            "visitor_fga": 1,
            "visitor_fgp": 1,
            "visitor_ftm": 1,
            "visitor_fta": 1,
            "visitor_ftp": 1,
            "visitor_tpm": 1,
            "visitor_tpa": 1,
            "visitor_tpp": 1,
            "visitor_offReb": 1,
            "visitor_defReb": 1,
            "visitor_totReb": 1,
            "visitor_assists": 1,
            "visitor_pFouls": 1,
            "visitor_steals": 1,
            "visitor_turnovers": 1,
            "visitor_blocks": 1,
            "visitor_plusMinus": 1,
            "home_points_stats": 1,
            "home_fgm": 1,
            "home_fga": 1,
            "home_fgp": 1,
            "home_ftm": 1,
            "home_fta": 1,
            "home_ftp": 1,
            "home_tpm": 1,
            "home_tpa": 1,
            "home_tpp": 1,
            "home_offReb": 1,
            "home_defReb": 1,
            "home_totReb": 1,
            "home_assists": 1,
            "home_pFouls": 1,
            "home_steals": 1,
            "home_turnovers": 1,
            "home_blocks": 1,
            "home_plusMinus": 1,
            "Over/Under": "$odds_data.Over/Under"

        }
    },
    {
        "$out": "Model_Ready"
    }
]

# Execute the aggregation pipeline
db.joined_games.aggregate(pipeline)

# Close MongoDB connection
mongo.close()


Existing 'Model_Ready' collection dropped.


In [26]:
# 13 Deletes any blanks for the Over/Under column in Model_Ready collection
 
# Connect to MongoDB
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]

# Define the filter condition to match documents where Over/Under is null
filter_condition = {"Over/Under": {"$exists": False}}

# Delete documents matching the filter condition from the Model_Ready collection
result = db.Model_Ready.delete_many(filter_condition)

# Print the number of documents deleted
print(f"Deleted {result.deleted_count} documents where Over/Under is null.")

# Close MongoDB connection
mongo.close()

Deleted 332 documents where Over/Under is null.


In [27]:
# 14 Create column for result of over/under line

# Connect to MongoDB
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]

# Iterate over documents in the Model_Ready collection
for document in db.Model_Ready.find({}):
    # Calculate the sum of visitor_points_game and home_points_game
    total_points = document["visitor_points_game"] + document["home_points_game"]
    
    # Check if total_points is greater than Over/Under
    if total_points > document["Over/Under"]:
        result = "Over"
    else:
        result = "Under"
    
    # Update the document with the calculated result
    db.Model_Ready.update_one(
        {"_id": document["_id"]},
        {"$set": {"Result": result}}
    )

# Close MongoDB connection
mongo.close()

In [37]:
# 15 Creating CSV of Model_Ready data

mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]
model_collection = db['Model_Ready']

# Convert the Model_Ready collection to a DataFrame
cursor = model_collection.find({})
model_df = pd.DataFrame(list(cursor))

# Close MongoDB connection
mongo.close()

# Save the DataFrame to a CSV file
model_df.to_csv("../data_files/Model_Ready.csv", index=False)

# Optionally, display the first few rows of the DataFrame
print(model_df.head())

                        _id                  _id_game  game_id  \
0  665d24b201b752cf6d7cfd0b  665b7e8c4e5b87fcb26da3c4     8787   
1  665d24b201b752cf6d7cfd0c  665b7e8c4e5b87fcb26da3c5     8788   
2  665d24b201b752cf6d7cfd0d  665b7e8c4e5b87fcb26da3c6     8789   
3  665d24b201b752cf6d7cfd0e  665b7e8c4e5b87fcb26da3c7     8790   
4  665d24b201b752cf6d7cfd0f  665b7e8c4e5b87fcb26da3c8     8791   

                 date_start           visitor_name visitor_nickname  \
0  2021-10-19T23:30:00.000Z          Brooklyn Nets             Nets   
1  2021-10-20T02:00:00.000Z  Golden State Warriors         Warriors   
2  2021-10-20T23:00:00.000Z         Indiana Pacers           Pacers   
3  2021-10-20T23:00:00.000Z          Chicago Bulls            Bulls   
4  2021-10-20T23:30:00.000Z         Boston Celtics          Celtics   

  visitor_code  home_id           home_name home_nickname  ... home_defReb  \
0          BKN       21     Milwaukee Bucks         Bucks  ...        41.0   
1          GSW      